In [2]:
import re
import math
import pickle
import random
import pandas as pd

from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate
from googletrans import Translator

ModuleNotFoundError: No module named 'indic_transliteration'

In [1]:
# Data List
nums = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,30,40,50,60,70,80,90,100]
hindiN = ['शून्य','एक','दो','तीन','चार','पांच','छह','सात ','आठ','नौ','दस','ग्यारह','बारह','तेरह','चौदह','पन्द्रह','सोलह','सत्रह','अठारह','उन्नीस','बीस','तीस','चालीस','पचास','साठ','सत्तर','अस्सी','नब्बे','एक सौ']

letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
hindiL = ['ए', 'बी', 'सी', 'डी', 'ई', 'एफ', 'जी', 'एच', 'आई', 'जे', 'के', 'एल', 'एम','एन', 'ओ', 'पी', 'क्यू', 'आर', 'एस', 'टी', 'यू','वी', 'डब्ल्यू', 'एक्स', 'वाई', 'जेड']

hindi_letters ={letters[i]:hindiL[i] for i in range(len(letters))} 
hindi_nums = {nums[i]:hindiN[i] for i in range(len(nums))}

In [2]:
translator = Translator()
data = ['Tamil Nadu', 'Maharashtra', 'Trivandrum', 'Salem', 'Trichy', 'Coimbatore', 'Pondicherry']
for place in data:
    text = translator.translate(place, src='en', dest='hi')
    print(text.text, end='  ')
    print(transliterate(place, sanscript.ITRANS, sanscript.DEVANAGARI))

# So we need to process the text before conversion as it is difficult to convert directly
print(translator.translate('104,2ND AGRAHARAM,NEAR OLD BUS STAND', src='en', dest='hi').text)

NameError: name 'Translator' is not defined

In [3]:
hospitals = pd.read_excel('./drive/My Drive/HospitalData.xls')
hospitals.drop(columns=['SL NO'],axis=1,inplace=True)
hospitals.head(n = 4)
col = list(hospitals.columns)
print(col)

NameError: name 'pd' is not defined

In [4]:
sentence_dict = dict()
sentence_dict['HOSPITAL NAME'] = ['अस्पताल का नाम $NAME है| ']
sentence_dict['state_city'] = ['यह अस्पताल $STATE राज्य के $CITY नगर में स्थित है। ', '$STATE राज्य के अंतर्गत आने वाले कई अस्पतालों में से यह एक है। यह अस्पताल $CITY नगर में स्थित है।']
sentence_dict['ZONE'] = ['यह अस्पताल $ZONE जोन के अंतर्गत आता है। ','यह अस्पताल $ZONE जोन में आने वाले प्रसिद्ध अस्पतालों में से एक है। ', '$ZONE जोन में आने वाले कई नामी अस्पतालों में इसकी गिनती की जाती है। ']
sentence_dict['CATEGORY'] = ['यह एक $CATEGORY का अस्पताल है। ', 'यह $CATEGORY में आने वाले कई नामी अस्पतालों में से एक है। ', 'यह अस्पताल $CATEGORY  के लिए मशहूर है। ']
sentence_dict['Hospital ID'] = ['इस अस्पताल का क्रमांक $HOSPITALID है। ']
sentence_dict['ADDRESS'] = ['इसका पता है : $ADDRESS । ', 'यह $ADDRESS में स्थित है। ']
sentence_dict['PINCODE'] = ['इसका पिन कोड है : $PINCODE । ']
sentence_dict['PHONE 1'] = ['इसका फ़ोन नंबर है : $PHONE । ', 'इस अस्पताल से संपर्क करने के लिए $PHONE फ़ोन नंबर का प्रयोग करें। ', 'इस अस्पताल की सभी सुविधाओं के बारे में जानकारी प्राप्त करने क लिए $PHONE को डायल करें। ']

should_translate = [1, 1, 1, 1, 1, 0, 1, 0, 0]

In [5]:
'''
Replace $ with the given word
'''
def completed_sentence(sentence, substitute):
    words = sentence.split()
    new_words = []
    for word in words:
        if word[0] == '$':
            new_words.append(str(substitute))
        else:
            new_words.append(str(word))
    return ' '.join(new_words)
'''
current_row is a dictionary with keys from the data frame
'''
def create_article(current_row):
    article = ""
    state = ""
    city = ""
    trans = 0
    for key in current_row.keys():
        print(key)
        if should_translate[trans] == 1:
            translated = translator.translate(str(current_row[key]), src='en', dest='hi')
            current_row[key] = translated.text
        trans += 1
        if key == 'STATE':
            state = current_row[key]
        elif key == 'CITY':
            city = current_row[key]
        else:
            ind = random.randint(0, len(sentence_dict[key]) - 1)
            article += completed_sentence(sentence_dict[key][ind], current_row[key])
    
    ind = random.randint(0, len(sentence_dict['state_city']) - 1)
    words = sentence_dict['state_city'][ind]
    words = words.split()
    new_words = []
    first = False
    for word in words:
        if word[0] == '$':
            if not first:
                new_words.append(state)
                first = True
            else:
                new_words.append(city)
        else:
            new_words.append(word)
    article += ' '.join(new_words)
    return article

In [6]:
print(hospitals.shape)
hospitals.info()
pd.set_option('display.max_columns', 50)

NameError: name 'hospitals' is not defined

In [ ]:
hospitals[:][:4]

In [7]:
position = dict()
position['1st'] = 'first'
position['2nd'] = 'second'
position['3rd'] = 'third'
position['4th'] = 'fourth'
position['6th'] = 'sixth'
position['7th'] = 'seventh'
position['8th'] = 'eighth'
position['9th'] = 'ninth'
'''
process for better translation
'''
def process(data):
    data = data.split()
    new_data = []
    for d in data:
        d = str(d)
        d = d.lower()
        if d in position:
            new_data.append(position[d])
        else:
            new_data.append(d)
    return ' '.join(new_data)
            
for i in range(4):
    current_row = dict()
    for k in col:
        current_row[k] = process(str(hospitals[k][i]))
    article = create_article(current_row)
    print(article)


NameError: name 'col' is not defined